In [ ]:
import matplotlib.pyplot as plt

from time import time
import networkx as nx
import pickle
import numpy as np
import os
import pandas as pd

#import helper libraries
from dynamicgem.utils  import graph_util, plot_util, dataprep_util
from dynamicgem.evaluation import visualize_embedding as viz
from dynamicgem.visualization import plot_dynamic_sbm_embedding
from dynamicgem.evaluation import evaluate_graph_reconstruction as gr
from dynamicgem.graph_generation import dynamic_SBM_graph as sbm

#import the methods
from dynamicgem.embedding.ae_static    import AE
from dynamicgem.embedding.dynAE        import DynAE
from dynamicgem.embedding.dynRNN       import DynRNN
from dynamicgem.embedding.dynAERNN     import DynAERNN
from sklearn.manifold import TSNE
import seaborn as sns

In [2]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# You can also import metrics to aggregate them
from sklearn.metrics import precision_recall_fscore_support

In [3]:
# Disable XLA
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

## Constructing temporal random graphs using dynamic stochastic block model with diminishing community.

In [5]:
# Parameters for Stochastic block model graph
# Todal of 1000 nodes
node_num           = 200
# Test with two communities
community_num      = 2
# At each iteration migrate 10 nodes from one community to the another
node_change_num    = 2
# Length of total time steps the graph will dynamically change
length             = 7
# output directory for result
outdir = './output'
intr='./intermediate'
if not os.path.exists(outdir):
    os.mkdir(outdir)
if not os.path.exists(intr):
    os.mkdir(intr)  
testDataType = 'sbm_cd'
#Generate the dynamic graph
dynamic_sbm_series = list(sbm.get_community_diminish_series_v2(node_num, 
                                                          community_num, 
                                                          length, 
                                                          1, #comminity ID to perturb
                                                          node_change_num))

community_size [108  92]
Step 0
Migrating Nodes
[126, 194]
Node 126 change from community 1 to 0
Node 194 change from community 1 to 0
Dynamically changed nodes
[144, 133]
Step 1
Migrating Nodes
[144, 133]
Node 144 change from community 1 to 0
Node 133 change from community 1 to 0
Dynamically changed nodes
[178, 171]
Step 2
Migrating Nodes
[178, 171]
Node 178 change from community 1 to 0
Node 171 change from community 1 to 0
Dynamically changed nodes
[138, 108]
Step 3
Migrating Nodes
[138, 108]
Node 138 change from community 1 to 0
Node 108 change from community 1 to 0
Dynamically changed nodes
[128, 136]
Step 4
Migrating Nodes
[128, 136]
Node 128 change from community 1 to 0
Node 136 change from community 1 to 0
Dynamically changed nodes
[158, 181]
Step 5
Migrating Nodes
[158, 181]
Node 158 change from community 1 to 0
Node 181 change from community 1 to 0
Dynamically changed nodes
[161, 135]


In [6]:
graphs = [g[0] for g in dynamic_sbm_series]


In [7]:
# Loop through each graph and print its basic information
for i, graph in enumerate(graphs):
    print(f"Graph {i+1}:")
    print(f"Number of nodes: {graph.number_of_nodes()}")
    print(f"Number of edges: {graph.number_of_edges()}")
    print(f"Is directed: {graph.is_directed()}")
    print("-" * 40)

Graph 1:
Number of nodes: 200
Number of edges: 4364
Is directed: True
----------------------------------------
Graph 2:
Number of nodes: 200
Number of edges: 4334
Is directed: True
----------------------------------------
Graph 3:
Number of nodes: 200
Number of edges: 4282
Is directed: True
----------------------------------------
Graph 4:
Number of nodes: 200
Number of edges: 4214
Is directed: True
----------------------------------------
Graph 5:
Number of nodes: 200
Number of edges: 4155
Is directed: True
----------------------------------------
Graph 6:
Number of nodes: 200
Number of edges: 4131
Is directed: True
----------------------------------------
Graph 7:
Number of nodes: 200
Number of edges: 4089
Is directed: True
----------------------------------------


In [9]:
# Example for the first graph
edges = list(graphs[0].edges(data=True))
df = pd.DataFrame(edges, columns=["Source", "Target", "Attributes"])
df['Weight'] = df['Attributes'].apply(lambda x: x.get('weight', None))
print(df)

      Source  Target Attributes Weight
0          0       2         {}   None
1          0       9         {}   None
2          0      11         {}   None
3          0      13         {}   None
4          0      15         {}   None
...      ...     ...        ...    ...
4359     199     189         {}   None
4360     199     191         {}   None
4361     199     192         {}   None
4362     199     193         {}   None
4363     199     198         {}   None

[4364 rows x 4 columns]


## Constructing temporal graphs from provided adjacency matrices

In [10]:
def get_graphs_from_adj_matrices(adj_matrices):
    """
    This function takes a list of temporal adjacency matrices, interprets their contents as graphs,
    and creates a list of directed graphs (DiGraph) using networkx. It returns the list of graphs
    along with the total number of graphs.
    
    Parameters:
    - adj_matrices: A list of 2D numpy arrays or lists, where each 2D array represents an adjacency
      matrix of a directed graph.
    
    Returns:
    - graphs: A list of networkx DiGraph objects created from the adjacency matrices.
    - length: The number of graphs created (which is the same as the length of the input list).
    """
    
    graphs = []
    
    # Iterate over each adjacency matrix in the list
    for adj_matrix in adj_matrices:
        G = nx.DiGraph()
        
        # Assuming adj_matrix is a 2D numpy array or a 2D list
        rows, cols = len(adj_matrix), len(adj_matrix[0])
        
        # Iterate over all elements of the adjacency matrix
        for i in range(rows):
            for j in range(cols):
                if adj_matrix[i][j] != 0:  # If there is an edge (non-zero weight)
                    G.add_edge(i, j, weight=adj_matrix[i][j])
        
        graphs.append(G)
    
    # Return the list of graphs and the total number of graphs
    return graphs, len(graphs)

In [11]:
# adj_matrices = [np.random.randint(0, 2, size=(200, 200)) for _ in range(20)]

adj_time_list = pd.read_pickle(f'../../Dataset/adj_time_list_hippocampus_rat.pkl')

In [12]:
threshold = 0 # choose accordingly 
adj_matrices = [np.abs(np.where(np.abs(connectivity_matrix.toarray()) < threshold, 0, connectivity_matrix.toarray())) for connectivity_matrix in adj_time_list]


In [13]:
graphs, len_graphs = get_graphs_from_adj_matrices(adj_matrices)

In [14]:
len_graphs

85

# Embedding Algorithms

In [15]:
# parameters for the dynamic embedding
# dimension of the embedding
embs_save_folder = "../../Dataset"
dim_emb  = 8
lookback = 2
tsne = TSNE(n_components=2, perplexity=40, n_iter=1000, random_state=4)


## Static AE: 
This method uses deep autoencoder to learn the representation of each node in the graph

![Alt text](./baseline_images/DynGEM.png)

Goyal, P., Kamra, N., He, X., & Liu, Y. (2018). DynGEM: Deep Embedding Method for Dynamic Graphs. arXiv preprint arXiv:1805.11273.

In [34]:
#AE Static
embedding = AE(d            = dim_emb, 
                 beta       = 5, 
                 nu1        = 1e-6, 
                 nu2        = 1e-6,
                 K          = 3, 
                 n_units    = [500, 300, ],
                 n_iter     = 100, 
                 xeta       = 1e-4,
                 n_batch    = 50,
                 modelfile  = ['./intermediate/enc_modelsbm.json',
                             './intermediate/dec_modelsbm.json'],
                 weightfile = ['./intermediate/enc_weightssbm.hdf5',
                             './intermediate/dec_weightssbm.hdf5'])
embs_AE  = []
t1 = time()
#ae static
for temp_var in range(len_graphs):
    print('window = ', temp_var)
    emb, _= embedding.learn_embeddings(graphs[temp_var])
    embs_AE.append(emb)


window =  0
No GPU found.
Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 868.4406  
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 420.1784
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 381.0718
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 290.3867
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 662.2999
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 339.8762
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 181.5526
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 190.8977
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 275.4124
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 637.3544
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

## Dynamic AE: 
This method models the interconnection of nodes within and acrosstime using multiple fully connected layers. It extends Static AE for dynamic graphs. 
This autoencoder model uses multiple fully connected layers for both encoder and decoder to capture highly non-linear interactions between nodes at each time step and across multiple time steps. The input to this model at each node is the neighborhood vector of that node.

![Alt text](./baseline_images/DynamicAE.png)

Goyal, P., Chhetri, S. R., & Canedo, A. (2018). dyngraph2vec: Capturing Network Dynamics using Dynamic Graph Representation Learning. arXiv preprint arXiv:1809.02657.


In [36]:
#dynAE
embedding= DynAE(d           = dim_emb,
                 beta           = 5,
                 n_prev_graphs  = lookback,
                 nu1            = 1e-6,
                 nu2            = 1e-6,
                 n_units        = [500, 300,],
                 rho            = 0.3,
                 n_iter         = 100,
                 xeta           = 1e-4,
                 n_batch        = 50,
                 modelfile      = ['./intermediate/enc_model_dynAE.json', 
                                   './intermediate/dec_model_dynAE.json'],
                 weightfile     = ['./intermediate/enc_weights_dynAE.hdf5', 
                                   './intermediate/dec_weights_dynAE.hdf5'],
                 savefilesuffix = "testing" )
embs_DynAE = []
t1 = time()
for temp_var in range(lookback+1, len_graphs+1):
    print('window = ', temp_var)
    emb, _ = embedding.learn_embeddings(graphs[:temp_var])
    embs_DynAE.append(emb)


window =  3
Epoch 1/10
# of batches: 18
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 69.7349
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 46.6549
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 43.4157
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 39.8069
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 37.9017
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 36.6205
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 35.9070
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 35.0971
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 34.7860
Epoch 10/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 34.9348
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2

## Dynamic RNN: 
This method uses sparsely connected Long Short Term Memory(LSTM) networks to learn the embedding. This model uses LSTM networks as both encoder and decoder to capture the long-term dependencies in dynamic graphs. Comparing to DynAE, the number of parameters is reduced and the model is capable of learning complex temporal patterns more efficiently. The input to this model at each node is the neighborhood vector of that node.This model uses LSTM networks as both encoder and decoder to capture the long-term dependencies in dynamic graphs. Comparing to DynAE, the number of parameters is reduced and the model is capable of learning complex temporal patterns more efficiently. The input to this model at each node is the neighborhood vector of that node.

![Alt text](./baseline_images/DynamicAERNN.png)

Goyal, P., Chhetri, S. R., & Canedo, A. (2018). dyngraph2vec: Capturing Network Dynamics using Dynamic Graph Representation Learning. arXiv preprint arXiv:1809.02657.

In [ ]:
#dynRNN
embedding= DynRNN(d        = dim_emb,
                beta           = 5,
                n_prev_graphs  = lookback,
                nu1            = 1e-6,
                nu2            = 1e-6,
                n_enc_units    = [500,300],
                n_dec_units    = [500,300],
                rho            = 0.3,
                n_iter         = 10,
                xeta           = 1e-3,
                n_batch        = 50,
                modelfile      = ['./intermediate/enc_model_dynRNN.json', 
                                  './intermediate/dec_model_dynRNN.json'],
                weightfile     = ['./intermediate/enc_weights_dynRNN.hdf5', 
                                  './intermediate/dec_weights_dynRNN.hdf5'],
                savefilesuffix = "testing"  )
embs_DynRNN = []
t1 = time()
for temp_var in range(lookback+1, len_graphs+1):
    print('window = ', temp_var)
    emb, _ = embedding.learn_embeddings(graphs[:temp_var])
    embs_DynRNN.append(emb)

## Dynamic AERNN: 
This method uses a fully connected encoder to initially ac-quire low dimensional hidden representation and feeds this representation into LSTMs to capture network dynamics.  Instead of passing the input adjacency matrices into LSTM, DynAERNN uses a fully connected encoder to initially acquire low dimensional hidden representations and then pass them as the input of LSTM to learn the embedding. The decoder of this model is a fully connected network similar to DynAE. The input to this model at each node is the neighborhood vector of that node.

![Alt text](./baseline_images/DynamicAERNN.png)

Goyal, P., Chhetri, S. R., & Canedo, A. (2018). dyngraph2vec: Capturing Network Dynamics using Dynamic Graph Representation Learning. arXiv preprint arXiv:1809.02657.

In [ ]:
#dynAERNN
embedding = DynAERNN(d   = dim_emb,
            beta           = 5,
            n_prev_graphs  = lookback,
            nu1            = 1e-6,
            nu2            = 1e-6,
            n_aeunits      = [500, 300],
            n_lstmunits    = [500,dim_emb],
            rho            = 0.3,
            n_iter         = 100,
            xeta           = 1e-3,
            n_batch        = 100,
            modelfile      = ['./intermediate/enc_model_dynAERNN.json', 
                              './intermediate/dec_model_dynAERNN.json'],
            weightfile     = ['./intermediate/enc_weights_dynAERNN.hdf5', 
                              './intermediate/dec_weights_dynAERNN.hdf5'],
            savefilesuffix = "testing")

embs_DynAERNN = []
t1 = time()
for temp_var in range(lookback+1, len_graphs+1):
    print('window = ', temp_var)
    emb, _ = embedding.learn_embeddings(graphs[:temp_var])
    embs_DynAERNN.append(emb)